#Boltzmann Machine

##Downloading the dataset

###ML-100K

In [ ]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2020-06-08 15:18:04--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  15.6MB/s    in 0.3s    

2020-06-08 15:18:04 (15.6 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

###ML-1M

In [ ]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2020-06-08 15:18:16--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  2.44MB/s    in 2.3s    

2020-06-08 15:18:19 (2.44 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
ml-100k  ml-100k.zip  ml-1m  ml-1m.zip	sample_data


##Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset


In [2]:
movies = pd.read_csv('movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users =  pd.read_csv('users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings =  pd.read_csv('ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

## Preparing the training set and the test set


In [3]:
training_set = pd.read_csv('u1.base', delimiter = '\t') 
training_set = np.array(training_set, dtype = 'int') # Transforming the dataframe to np array
test_set = pd.read_csv('u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int') # Transforming the dataframe to np array

## Getting the number of users and movies


In [4]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0]))) # The number of users that we have in the test set and the training set which is the maximum user ID from the training set
# and the test set 

nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1]))) # The number of users that we have in the test set and the training set which is the maximum user ID 
# from the training set and the test set 

## Converting the data into an array with users in lines and movies in columns


In [5]:
def convert(data):
  new_data = []

  for id_users in range(1, nb_users+1):
    id_movies = data[:, 1] [data[:, 0] == id_users] # This a syntax of a conditional statment in pyhton
    
    id_ratings = data[:, 2] [data[:, 0] == id_users] # This a syntax of a conditional statment in pyhton

    ratings = np.zeros(nb_movies) # Here we are initializing a new lits of zeros with the size of all the movies that we have so we can replace the zeros
    # with the rating if it is there

    ratings[id_movies - 1] = id_ratings # Here we are trying to build the ratings array by finding the matches for it in the id_ratings so if there is a
    # raing for the movie id mentioned then the ratings will be transfered or equaled otherwise it will still be 0

    new_data.append(list(ratings)) # appending the ratings of all the movies in the users list that will result in a 2d list
  
  return new_data

training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors


In [6]:
#Remember the tesnor is a multi-dimensional matrix with one type in it 

training_set = torch.FloatTensor(training_set) # We are converting the 2d list of training set into a 2d Floattensor so we can build the architecture of the boltzmann network
# so we can do the training and the building much effecient with tensors

test_set = torch.FloatTensor(test_set) # We are converting the 2d list of test set into a 2d Floattensor so we can build the architecture of the boltzmann network
# so we can do the training and the building much effecient with tensors

## Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)


In [7]:
training_set[training_set == 0] = -1# we want to change all the ratings to either 0 or 1 so we have to give another category for the missing data whcih is -1
training_set[training_set == 1] = 0 # we want to change all the ratings to either 0 or 1 so if the user put a rating 1 for the movie that indicates that the user
# did not like the movie
training_set[training_set == 2] = 0 # we want to change all the ratings to either 0 or 1 so if the user put a rating 2 for the movie that indicates that the user
# did not like the movie we couldn't do the previous this step and the previous step at once becuase FloatTensors do not accept or statements

training_set[training_set >= 3] = 1 # we want to change all the ratings to either 0 or 1 so if the user put a rating 3 or higher for the movie that indicates 
# that the user did not like the movie

test_set[test_set == 0] = -1# we want to change all the ratings to either 0 or 1 so we have to give another category for the missing data whcih is -1
test_set[test_set == 1] = 0 # we want to change all the ratings to either 0 or 1 so if the user put a rating 1 for the movie that indicates that the user
# did not like the movie
test_set[test_set == 2] = 0 # we want to change all the ratings to either 0 or 1 so if the user put a rating 2 for the movie that indicates that the user
# did not like the movie we couldn't do the previous this step and the previous step at once becuase FloatTensors do not accept or statements

test_set[test_set >= 3] = 1 # we want to change all the ratings to either 0 or 1 so if the user put a rating 3 or higher for the movie that indicates 
# that the user did not like the movie

## Creating the architecture of the Neural Network


In [8]:
class RBM(): # Here we making a class that will be the essense of building the architecture of the Restricted Boltzmann Machine
  
  
  def __init__(self, number_of_visible_nodes, number_of_hidden_nodes): # Initializng the class with two elements

    self.weights = torch.randn(number_of_hidden_nodes, number_of_visible_nodes) # Initializng the weights of both the hidden and the visible nodes in a normal distribution 
    # This will give as the weights as a matrix of dimension hidden_nodes x visible_nodes which all the possible connection between the nodes

    self.bias_for_hidden_nodes = torch.randn(1, number_of_hidden_nodes) # Initializng the weights of both the hidden and the visible nodes in a normal distribution 
    # This will give as the weights as a matrix of dimension hidden_nodes x visible_nodes which all the possible connection between the nodes

    self.bias_for_visible_nodes = torch.randn(1, number_of_visible_nodes)# Initializng the weights of both the hidden and the visible nodes in a normal distribution 
    # This will give as the weights as a matrix of dimension hidden_nodes x visible_nodes which all the possible connection between the nodes

  
  def sample_hidden(self, x):
    weight_x_node = torch.mm(x, self.weights.t()) # Here we are multiplying the output of the node by the weight to determine then if the node is activated or not
    
    activation = weight_x_node +self.bias_for_hidden_nodes.expand_as(weight_x_node) # Here we are buidling the activation function of all the hidden nodes 
    
    p_h_given_v = torch.sigmoid(activation) # Here we are simply getting the probability of activation of the hidden node  from the previous node which
    # is given to be visible using the sigmoid function and the value of the activation function done on all the hidden nodes
    
    return p_h_given_v, torch.bernoulli(p_h_given_v) # Here we are returning the probabilty calculated before and then the bernoulli distribution of the probabilities
    # either 0 or 1

  
  def sample_visible(self, y):
    weight_y_node = torch.mm(y, self.weights) # Here we are multiplying the output of the node by the weight to determine then if the node is activated or not
    
    activation = weight_y_node +self.bias_for_visible_nodes.expand_as(weight_y_node) # Here we are buidling the activation function of all the visible nodes 
    
    p_v_given_h = torch.sigmoid(activation) # Here we are simply getting the probability of activation of the visible node  from the previous node which
    # is given to be hidden using the sigmoid function and the value of the activation function done on all the visible nodes
    
    return p_v_given_h, torch.bernoulli(p_v_given_h) # Here we are returning the probabilty calculated before and then the bernoulli distribution of the probabilities
    # either 0 or 1
  

  def train(self, input_vector, visible_nodes_after_k_iterations, prob_of_hidden_given_input_vector, prob_of_hidden_given_visible_nodes_after_k_iterations):
    self.weights += (torch.mm(input_vector.t(), prob_of_hidden_given_input_vector) - torch.mm(visible_nodes_after_k_iterations.t(), prob_of_hidden_given_visible_nodes_after_k_iterations)).t()
    
    self.bias_for_visible_nodes += torch.sum((input_vector - visible_nodes_after_k_iterations), 0)

    self.bias_for_hidden_nodes += torch.sum((prob_of_hidden_given_input_vector - prob_of_hidden_given_visible_nodes_after_k_iterations), 0)


In [9]:
number_of_visible_nodes = len(training_set[0]) # The number of movies
number_of_hidden_nodes = 100 # The number of nodes we need our predictions to depend on
batch_size = 100 # The number of data we need to train our data on every epoch

rbm = RBM(number_of_hidden_nodes = number_of_hidden_nodes, number_of_visible_nodes = number_of_visible_nodes) 


## Training the RBM


In [10]:
number_of_epochs = 100

for epoch in range(1, number_of_epochs + 1):
  train_loss = 0
  counter = 0.
  for id_user in range(0, nb_users- batch_size, batch_size):
    visible_node_after_k_iteration = training_set[id_user : id_user + batch_size]
    input_vector = training_set[id_user : id_user + batch_size]
    prob_of_hidden_given_input_vector, _ = rbm.sample_hidden(input_vector)
    for k in range(10):
      _, hidden_node_after_k_iteration = rbm.sample_hidden(visible_node_after_k_iteration)
      _, visible_node_after_k_iteration = rbm.sample_visible(hidden_node_after_k_iteration)
      visible_node_after_k_iteration[input_vector < 0] = input_vector[input_vector<0]
    prob_of_hidden_given_after_k_iteration, _ = rbm.sample_hidden(visible_node_after_k_iteration)
    rbm.train(input_vector, visible_node_after_k_iteration, prob_of_hidden_given_input_vector, prob_of_hidden_given_after_k_iteration)
    train_loss += torch.mean(torch.abs(input_vector[input_vector >= 0] - visible_node_after_k_iteration[input_vector>=0]))
    counter += 1
  print("Epoch: "+str(epoch)+" loss: "+str(train_loss/counter))

Epoch: 1 loss: tensor(0.3443)
Epoch: 2 loss: tensor(0.2507)
Epoch: 3 loss: tensor(0.2468)
Epoch: 4 loss: tensor(0.2505)
Epoch: 5 loss: tensor(0.2484)
Epoch: 6 loss: tensor(0.2479)
Epoch: 7 loss: tensor(0.2489)
Epoch: 8 loss: tensor(0.2480)
Epoch: 9 loss: tensor(0.2484)
Epoch: 10 loss: tensor(0.2457)
Epoch: 11 loss: tensor(0.2466)
Epoch: 12 loss: tensor(0.2478)
Epoch: 13 loss: tensor(0.2498)
Epoch: 14 loss: tensor(0.2462)
Epoch: 15 loss: tensor(0.2496)
Epoch: 16 loss: tensor(0.2459)
Epoch: 17 loss: tensor(0.2461)
Epoch: 18 loss: tensor(0.2457)
Epoch: 19 loss: tensor(0.2461)
Epoch: 20 loss: tensor(0.2469)
Epoch: 21 loss: tensor(0.2465)
Epoch: 22 loss: tensor(0.2459)
Epoch: 23 loss: tensor(0.2482)
Epoch: 24 loss: tensor(0.2443)
Epoch: 25 loss: tensor(0.2476)
Epoch: 26 loss: tensor(0.2472)
Epoch: 27 loss: tensor(0.2470)
Epoch: 28 loss: tensor(0.2450)
Epoch: 29 loss: tensor(0.2456)
Epoch: 30 loss: tensor(0.2462)
Epoch: 31 loss: tensor(0.2462)
Epoch: 32 loss: tensor(0.2467)
Epoch: 33 loss: t

## Testing the RBM


In [12]:
test_loss = 0
counter = 0.

for id_user in range(nb_users):
   visible_node_after_1_iteration = training_set[id_user : id_user + 1]
   target_vector = test_set[id_user : id_user + 1]
   
   if len(target_vector[target_vector>=0]) > 0:
     _, hidden_node_after_1_iteration = rbm.sample_hidden(visible_node_after_1_iteration)
     _, visible_node_after_1_iteration = rbm.sample_visible(hidden_node_after_1_iteration)
     test_loss += torch.mean(torch.abs(target_vector[target_vector >= 0] - visible_node_after_1_iteration[target_vector>=0]))
     counter += 1.

print("test loss: "+str(test_loss/counter))

test loss: tensor(0.2470)
